In [2]:
import pandas as pd
import matplotlib
import matplotlib.pyplot as plt

In [ ]:
# Create a dictionary that maps the run# to total ixode { run#: host_density }

paramfile = ""
paramd = {}
with open(paramfile, 'r') as file:
    for line in file:
        result = line.replace("\t",",").split(',')
        # attempt to add the density value to run, if it doesn't exist, create it
        try:
            paramd[float(result([0]))].append(float(result[4]))
        except KeyError:
            paramd[float(result[0])] = result[4]  # { run#: host_density }

for k, v in paramd.items():
    print(k,v)

In [ ]:
# Create a dictionary that maps the run# to total ixode { run#: cumulative_ixode }
ixode_count_dict = {}
df = pd.read_csv("", error_bad_lines=False)
for run in df.groupby('run'):
    current_df = run[1]
    ixode_count_dict[run[0]] = len(current_df['name'].unique())

In [ ]:
# Creating a data frame from the cumulative_ixode dict generated above
df_final = pd.DataFrame(ixode_count_dict.items(), columns=['run', 'total_ixode'])
df_final['host_density'] = 0
df_final.dtypes


In [ ]:
# Iterate through param dictionary and add the host density value to the associated run
for key, value in paramd.items():
    for i in range(len(value[0])):
        df_final.loc[df_final['run']==value[0][i], 'host_density'] = value[1][i]

In [20]:
df_test = pd.read_csv("../data/test")
df_test.dtypes

run               int64
host_density    float64
total_ixodes      int64
dtype: object

In [46]:
# Now we have a df with | 'run' | 'total_ixode' | 'host_density' |
# Create a dictionary with { host_density: agg_ixodes }
agg_ixodes_dict = {}
for density in df_test.groupby('host_density'):
    tmp_df = density[1]
    print(density[0])
    agg_ixodes_dict[density[0]] = tmp_df['total_ixodes'].agg('mean')


# should get 1880.75 for 0.4, 7898.5 for 0.7, and 24917 for 1.0


0.4
0.7
1.0


In [47]:
# Create a dataframe from the agg_ixodes_dict that we can the plot
df_test_final = pd.DataFrame(agg_ixodes_dict.items(), columns=['host_density', 'agg_ixodes'])
df_test_final.head()

,host_density,agg_ixodes
0,0.4,1880.75
1,0.7,7898.50
2,1.0,24917.00


In [48]:
matplotlib.use('Qt5Agg')
plt.plot(df_test_final['host_density'], df_test_final['agg_ixodes'])
plt.show(block=True)
